# [Optional] Introduction to Strands Agents

This optional module introduces the Strands Agents SDK for building AI agents. Complete this if you're new to agentic AI development before proceeding to the main Developer Journey labs.

## Learning Objectives

By the end of this notebook, you will be able to:
- Create a basic agent using the Strands Agents SDK
- Add built-in and custom tools to extend agent capabilities
- Understand the agent execution loop and metrics
- Deploy an agent to Amazon Bedrock AgentCore Runtime

**Duration**: ~30 minutes

## What is Strands Agents?

[Strands Agents](https://strandsagents.com/) is an open-source SDK for building AI agents with a model-driven approach. Instead of manually orchestrating complex workflows, Strands lets the model decide when and how to use tools.

### Core Components

| Component | Description |
|-----------|-------------|
| **Model** | The LLM that powers reasoning (e.g., Claude on Bedrock) |
| **System Prompt** | Instructions defining agent behavior and personality |
| **Tools** | Functions the agent can call to take actions |

### Agent Loop Architecture

```
                     +----------------------------------+
                     |         Strands Agent            |
+------------+       |                                  |       +------------+
| User Input | ----> |  +-------------------------+    | ----> |  Response  |
+------------+       |  |      Agent Loop         |    |       +------------+
                     |  |  1. Reason (LLM)        |    |
                     |  |  2. Select Tool         |    |
                     |  |  3. Execute Tool        |    |
                     |  |  4. Repeat or Respond   |    |
                     |  +-------------------------+    |
                     +----------------------------------+
```

The agent autonomously decides whether to respond directly or use tools based on the user's request.

---

## 1. Setup

Configure AWS credentials and model settings.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configuration
REGION = os.getenv('AWS_DEFAULT_REGION', 'us-east-1')
MODEL_ID = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"

print(f"Region: {REGION}")
print(f"Model: {MODEL_ID}")

---

## 2. Your First Agent

Create a simple agent with just a few lines of code. The agent uses Claude on Amazon Bedrock for reasoning.

In [ ]:
from strands import Agent

# Create a basic agent
agent = Agent(
    model=MODEL_ID,
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

# Invoke the agent
agent("What is Amazon Bedrock in one sentence?")

<div class="alert alert-block alert-info">
<b>Note:</b> The agent automatically handles the conversation with the model. You simply call the agent like a function with your input.
</div>

---

## 3. Adding Tools

Tools extend agent capabilities beyond text generation. The agent autonomously decides when to use tools based on the user's request.

### Built-in Tools

Strands provides common tools out of the box via the `strands-agents-tools` package:

In [ ]:
from strands import Agent
from strands_tools import calculator, current_time

# Create agent with built-in tools
agent_with_tools = Agent(
    model=MODEL_ID,
    tools=[calculator, current_time],
    system_prompt="You are a helpful assistant with access to a calculator and clock."
)

# The agent automatically uses the calculator tool
agent_with_tools("What is 15% of 250?")

In [ ]:
# The agent uses the current_time tool
agent_with_tools("What time is it now?")

### Custom Tools

Create domain-specific tools using the `@tool` decorator. This is essential for building agents that interact with your business systems.

In [ ]:
from strands import Agent, tool

@tool
def lookup_order(order_id: str) -> str:
    """Look up order status by order ID.
    
    Args:
        order_id: The order ID to look up (e.g., ORD-12345)
    
    Returns:
        Order status information
    """
    # Simulated order database
    orders = {
        "ORD-12345": {"status": "Shipped", "eta": "2 days", "carrier": "FedEx"},
        "ORD-67890": {"status": "Processing", "eta": "5 days", "carrier": "Pending"},
    }
    
    if order_id in orders:
        order = orders[order_id]
        return f"Order {order_id}: {order['status']}, ETA: {order['eta']}, Carrier: {order['carrier']}"
    return f"Order {order_id} not found"


@tool
def check_inventory(product_sku: str) -> str:
    """Check inventory for a product SKU.
    
    Args:
        product_sku: The product SKU to check
    
    Returns:
        Inventory status
    """
    # Simulated inventory
    inventory = {
        "SKU-KB-001": {"name": "Wireless Keyboard", "qty": 150, "warehouse": "Seattle"},
        "SKU-MS-002": {"name": "Gaming Mouse", "qty": 0, "warehouse": "N/A"},
    }
    
    if product_sku in inventory:
        item = inventory[product_sku]
        status = "In Stock" if item['qty'] > 0 else "Out of Stock"
        return f"{item['name']}: {status} ({item['qty']} units in {item['warehouse']})"
    return f"Product {product_sku} not found"


print("Custom tools defined: lookup_order, check_inventory")

<div class="alert alert-block alert-warning">
<b>Key Insight:</b> The docstring in your tool function is critical - it tells the model what the tool does and when to use it. Write clear, descriptive docstrings for better tool selection.
</div>

---

## 4. Customer Support Agent Example

Let's build a customer support agent similar to what we'll use in the main Developer Journey labs.

In [ ]:
support_agent = Agent(
    model=MODEL_ID,
    tools=[lookup_order, check_inventory],
    system_prompt="""You are a customer support agent for CloudCommerce.

Your responsibilities:
- Help customers check order status using the lookup_order tool
- Check product inventory using the check_inventory tool
- Be friendly, professional, and helpful

Always use the available tools to get accurate information.
If you cannot find information, apologize and offer alternatives."""
)

print("Customer support agent created with 2 tools")

In [ ]:
# Test order lookup
support_agent("Where is my order ORD-12345?")

In [ ]:
# Test inventory check
support_agent("Do you have the wireless keyboard SKU-KB-001 in stock?")

In [ ]:
# Test handling unknown order
support_agent("What's the status of order ORD-99999?")

---

## 5. Understanding Agent Execution

After running an agent, you can inspect execution metrics to understand what happened.

In [ ]:
result = support_agent("Check if SKU-MS-002 is available")

# Access metrics from the result
print("\n" + "=" * 50)
print("EXECUTION METRICS")
print("=" * 50)

if hasattr(result, 'metrics'):
    metrics = result.metrics
    if hasattr(metrics, 'accumulated_usage'):
        usage = metrics.accumulated_usage
        print(f"Input tokens:  {usage.get('inputTokens', 'N/A')}")
        print(f"Output tokens: {usage.get('outputTokens', 'N/A')}")
        print(f"Total tokens:  {usage.get('totalTokens', 'N/A')}")
else:
    print("Metrics not available in this response format")

---

## 6. Deploy to Amazon Bedrock AgentCore Runtime

Amazon Bedrock AgentCore Runtime provides a managed, serverless environment for running agents in production. This section shows how to deploy using the AgentCore Starter Toolkit CLI.

<div class="alert alert-block alert-info">
<b>Note:</b> The AgentCore deployment uses the <code>agentcore</code> CLI which handles container building, ECR push, and runtime creation. Run these commands in your terminal.
</div>

### What is AgentCore Runtime?

AgentCore Runtime is a fully managed service that hosts your AI agents:

1. **Packages your code** into a container
2. **Creates AWS resources** (IAM roles, ECR, etc.)
3. **Hosts your agent** in a serverless environment
4. **Provides an API endpoint** for invocation

### Key Benefits

| Feature | Description |
|---------|-------------|
| **Serverless** | No infrastructure to manage |
| **Auto-scaling** | Handles variable load automatically |
| **Session Isolation** | Each conversation runs in isolated context |
| **Framework Agnostic** | Works with Strands, LangGraph, CrewAI, and more |

### Step 1: Create Agent File

Create a file called `demo_agent.py` with your agent wrapped in `BedrockAgentCoreApp`:

In [ ]:
%%writefile demo_agent.py
from bedrock_agentcore import BedrockAgentCoreApp
from strands import Agent, tool

app = BedrockAgentCoreApp()

@tool
def lookup_order(order_id: str) -> str:
    """Look up order status by order ID."""
    orders = {
        "ORD-12345": {"status": "Shipped", "eta": "2 days", "carrier": "FedEx"},
        "ORD-67890": {"status": "Processing", "eta": "5 days", "carrier": "Pending"},
    }
    if order_id in orders:
        order = orders[order_id]
        return f"Order {order_id}: {order['status']}, ETA: {order['eta']}, Carrier: {order['carrier']}"
    return f"Order {order_id} not found"

agent = Agent(
    model="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    tools=[lookup_order],
    system_prompt="You are a helpful customer support agent."
)

@app.entrypoint
def invoke(payload):
    response = agent(payload.get("prompt", ""))
    return {"result": response.message}

if __name__ == "__main__":
    app.run()

In [ ]:
%%writefile requirements-demo.txt
bedrock-agentcore
strands-agents

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

# Initialize the Runtime
runtime = Runtime()
AGENT_NAME = "demo_agent"

# Configure the agent
runtime.configure(
    entrypoint="demo_agent.py",
    requirements_file="requirements-demo.txt",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    agent_name=AGENT_NAME
)

print("Agent configured")

### Step 3: Deploy and Test

Deploy the agent and test it:

In [ ]:
# Deploy to AgentCore Runtime
# This builds the container, pushes to ECR, and creates the runtime
runtime.launch(auto_update_on_conflict=True)

print("Deployment initiated...")

In [ ]:
import time

# Wait for agent to be ready
status = runtime.status()
agent_status = status.endpoint["status"]
terminal_states = {"READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"}

while agent_status not in terminal_states:
    print(f"Agent status: {agent_status}")
    time.sleep(10)
    status = runtime.status()
    agent_status = status.endpoint["status"]

print(f"Final status: {agent_status}")

In [ ]:
import json

# Test the deployed agent
response = runtime.invoke({"prompt": "Where is my order ORD-12345?"})

# Extract text from response
if isinstance(response, dict) and "response" in response:
    resp_data = response["response"]
    if isinstance(resp_data, list) and resp_data:
        parsed = json.loads(resp_data[0])
        text = parsed.get("result", {}).get("content", [{}])[0].get("text", str(parsed))
        print("Agent Response:")
        print(text)
else:
    print(response)

In [ ]:
# Clean up AgentCore resources using Python API
runtime.destroy(delete_ecr_repo=True)

# Clean up local files
import os
for f in ['demo_agent.py', 'requirements-demo.txt', '.bedrock_agentcore.yaml', 'Dockerfile', '.dockerignore']:
    if os.path.exists(f):
        os.remove(f)

print("Cleanup complete!")

---

## Summary

| Concept | Description |
|---------|-------------|
| **Agent** | Model + System Prompt + Tools |
| **Tools** | Functions the agent can call |
| **@tool decorator** | Creates custom tools from Python functions |
| **Agent Loop** | Reason -> Select Tool -> Execute -> Repeat/Respond |
| **AgentCore Runtime** | Managed serverless environment for production |

### Key Takeaways

1. **Simple API**: Create agents with just 3 components (model, prompt, tools)
2. **Tool-driven**: The model decides when and how to use tools
3. **Custom tools**: Use `@tool` decorator for domain-specific functionality
4. **Production-ready**: AgentCore Runtime handles scaling and infrastructure

---

**Next**: Continue to the main Developer Journey labs to build a complete customer support system with observability and evaluation.